In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import deap
import skopt
from tensorflow.keras.models import save_model

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
# Load and preprocess the data
def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)

    # Convert binary categorical features to 0 and 1
    binary_features = ['CryoSleep', 'VIP']
    df[binary_features] = df[binary_features].astype(bool).astype(int)

    # Extract components from 'Cabin' and treat them as separate features
    if 'Cabin' in df.columns:
        df[['Cabin_Deck', 'Cabin_Number', 'Cabin_Side']] = df['Cabin'].str.split('/', expand=True)
        df['Cabin_Side'] = df['Cabin_Side'].map({'P': 1, 'S': 0})
        df['Cabin_Number'] = pd.to_numeric(df['Cabin_Number'], errors='coerce')
        df.drop('Cabin', axis=1, inplace=True)

    # One-hot encode multi-category features
    multi_cat_features = ['HomePlanet', 'Destination']
    global one_hot_encoder  # Ensure the encoder is fit only once
    if is_train:
        one_hot_encoder = OneHotEncoder(drop='first')
        encoded_features = one_hot_encoder.fit_transform(df[multi_cat_features])
    else:
        encoded_features = one_hot_encoder.transform(df[multi_cat_features])

    encoded_feature_names = one_hot_encoder.get_feature_names_out(multi_cat_features)
    encoded_features_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_feature_names)
    df = pd.concat([df, encoded_features_df], axis=1)
    df.drop(multi_cat_features, axis=1, inplace=True)

    # Imputation and Scaling for numeric features
    numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin_Number', 'Cabin_Side']
    global imputer, scaler  # Ensure the imputer and scaler are fit only once
    if is_train:
        imputer = SimpleImputer(strategy='median')
        scaler = StandardScaler()
        df[numeric_features] = imputer.fit_transform(df[numeric_features])
        df[numeric_features] = scaler.fit_transform(df[numeric_features])
    else:
        df[numeric_features] = imputer.transform(df[numeric_features])
        df[numeric_features] = scaler.transform(df[numeric_features])

    if is_train:
        # Convert 'Transported' to integer (True=1, False=0) for modeling
        df['Transported'] = df['Transported'].astype(int)

    return df

train_df = preprocess_data('csv_files/train.csv', is_train=True)
test_df = preprocess_data('csv_files/test.csv', is_train=False)

# Prepare features and target for the model
features = [col for col in train_df.columns if col not in ['PassengerId', 'Name', 'Transported', 'Cabin_Deck']]
X = train_df[features]
y = train_df['Transported']

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handling class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_smote),
    y=y_train_smote)
class_weight_dict = dict(enumerate(class_weights))


# Function to create the model, now with an optimizer parameter
def create_model(input_shape, layers, activation, dropout_rate, learning_rate, optimizer_type='adam'):
    model = Sequential()
    model.add(Dense(layers[0], activation=activation, input_shape=(input_shape,)))
    model.add(Dropout(dropout_rate))
    
    for layer_size in layers[1:]:
        model.add(Dense(layer_size, activation=activation))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))

    # Choose the optimizer
    if optimizer_type == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        optimizer = Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Function to train and evaluate the model
def train_and_evaluate(model, X_train, y_train, X_val, y_val, batch_size, epochs, class_weight):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=(X_val, y_val), class_weight=class_weight, 
              callbacks=[early_stopping])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    return val_loss, val_acc

# Optimization loop
target_accuracy = 0.85
max_iterations = 40
current_iteration = 0
best_val_acc = 0
best_params = {'layers': [128, 128, 64], 'activation': 'tanh', 'dropout_rate': 0.3, 'learning_rate': 0.001}
# Initialization outside the loop
best_model = None

while current_iteration < max_iterations:
    # Use best parameters from previous iterations
    layers = best_params['layers']
    activation = best_params['activation']
    dropout_rate = best_params['dropout_rate']
    learning_rate = best_params['learning_rate']
    
    # Here you can decide to use SGD or Adam based on some condition, e.g., alternate between them
    if current_iteration % 2 == 0:
        optimizer_type = 'adam'
    else:
        optimizer_type = 'sgd'
    # Create and train the model
    model = create_model(input_shape=X_train_smote.shape[1], 
                         layers=layers, 
                         activation=activation, 
                         dropout_rate=dropout_rate, 
                         learning_rate=learning_rate,
                         optimizer_type=optimizer_type)

    val_loss, val_acc = train_and_evaluate(model, X_train_smote, y_train_smote, X_val, y_val, 
                                           batch_size=16, epochs=100, class_weight=class_weight_dict)

    print(f'Iteration {current_iteration} using {optimizer_type.upper()}: Validation Accuracy = {val_acc}')

    # Update best parameters if current model is better
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_params = {'layers': layers, 'activation': activation, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate ,'optimizer_type' : optimizer_type }
        best_model = model  
        print("New best model found and saved.")
    # Example of implementing learning rate decay
    if current_iteration > 0 and val_acc < best_val_acc:
        learning_rate *= 0.9  # decay learning rate by 10% if no improvement

# Example of adjusting model complexity based on performance 
    if val_acc < best_val_acc - 0.05:  # significant drop in performance
        layers = [max(l // 2, 32) for l in layers]  # reduce complexity
    elif val_acc > best_val_acc:
        layers = [min(l * 2, 512) for l in layers]  # increase complexity if showing improvement

    if best_val_acc >= target_accuracy:
        print("Target accuracy reached.")
        break

    current_iteration += 1

# Final evaluation
if best_val_acc < target_accuracy:
    print("Target accuracy not reached after maximum iterations.")
    print(f"Final Model Validation Accuracy: {best_val_acc}")
else:
    print(f"Final Model Validation Accuracy: {best_val_acc}")

model_enhanced = best_model
# Combine the original training and validation sets for final training
X_full, y_full = smote.fit_resample(X, y)

# Create and train the final model using the best parameters
final_model = create_model(
    input_shape=X_full.shape[1], 
    layers=best_params['layers'], 
    activation=best_params['activation'], 
    dropout_rate=best_params['dropout_rate'], 
    learning_rate=best_params['learning_rate'], 
    optimizer_type=best_params['optimizer_type']
)
# Train the final model
final_model.fit(X_full, y_full, batch_size=16, epochs=100, class_weight=class_weight_dict)
# Save the final model
final_model.save('model_enhanced.keras')


Epoch 1/100


/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step - accuracy: 0.7356 - loss: 0.5276 - val_accuracy: 0.7757 - val_loss: 0.4727
Epoch 2/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.7865 - loss: 0.4570 - val_accuracy: 0.7734 - val_loss: 0.4646
Epoch 3/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.7926 - loss: 0.4526 - val_accuracy: 0.7746 - val_loss: 0.4595
Epoch 4/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step - accuracy: 0.7838 - loss: 0.4500 - val_accuracy: 0.7752 - val_loss: 0.4580
Epoch 5/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.7878 - loss: 0.4525 - val_accuracy: 0.7752 - val_loss: 0.4695
Epoch 6/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.8003 - loss: 0.4279 - val_accuracy: 0.7757 - val_loss: 0.4586
Epoch 7/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.8011 - loss: 0.4350 - val_accuracy: 0.7832 - val_loss: 0.4538
Epoch 8/100
438/438 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step - accuracy: 0.7937 - loss: 0.4371 - val_

/Users/rahulnundlall/anaconda3/envs/ML/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


548/548 ━━━━━━━━━━━━━━━━━━━━ 1s 408us/step - accuracy: 0.7375 - loss: 0.5183
Epoch 2/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.7735 - loss: 0.4726
Epoch 3/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.7885 - loss: 0.4554
Epoch 4/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.7954 - loss: 0.4458
Epoch 5/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 399us/step - accuracy: 0.7816 - loss: 0.4521
Epoch 6/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.7830 - loss: 0.4497
Epoch 7/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - accuracy: 0.7904 - loss: 0.4405
Epoch 8/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.7917 - loss: 0.4322
Epoch 9/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.7897 - loss: 0.4336
Epoch 10/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.8013 - loss: 0.4304
Epoch 11/100
548/548 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7927 - loss: 0.4215
Epoch 12/100
548/548 ━━━━━━━━━━

In [34]:
from tensorflow.keras.models import load_model

# Load the best model
model_enhanced = load_model('model_enhanced.keras')
X_test = test_df[features]

# Assuming model_enhanced is your trained model with the best parameters
y_pred_test_proba = model_enhanced.predict(X_test)
y_pred_test = (y_pred_test_proba > 0.5).astype(int).flatten()

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': y_pred_test
})

# Convert predictions back to boolean (True/False) if necessary
submission_df['Transported'] = submission_df['Transported'].astype(bool)

# Save the submission file
submission_df.to_csv('enhanced_tensorflow_resultzz.csv', index=False)

134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step


In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# Define the space of hyperparameters to search
space = [
    Integer(32, 512, name='layer_size'),
    Real(0.01, 0.5, name='dropout_rate'),
    Real(1e-4, 1e-2, prior='log-uniform', name='learning_rate')
]

# Function to optimize (we aim to minimize 1 - accuracy)
@use_named_args(space)
def objective(**params):
    model = create_model(
        input_shape=X_train_smote.shape[1], 
        layers=[params['layer_size']] * 2,  # Example: using the same size for two layers
        activation='relu', 
        dropout_rate=params['dropout_rate'], 
        learning_rate=params['learning_rate']
    )
    val_loss, val_acc = train_and_evaluate(model, X_train_smote, y_train_smote, X_val, y_val, 
                                           batch_size=32, epochs=50, class_weight=class_weight_dict)
    return 1 - val_acc  # as we need to minimize

# Run Bayesian optimization
result = gp_minimize(objective, space, n_calls=20, random_state=0)

print(f"Best parameters: {result.x}")
print(f"Best validation accuracy: {1 - result.fun}")